<a href="https://colab.research.google.com/github/olaviinha/SloppyButchery/blob/main/StutteringButcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Stuttering Butcher <font color="#999" size="3">v 0.0.1<font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font size="4">Sloppy Butchery @</font> <a href="https://github.com/olaviinha/SloppyButchery" target="_blank"><font color="#999" size="4">Github</font></a><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font size="3" color="#999"><a href="https://inha.se" target="_blank"><font color="#999">O. Inha</font></a></font></font>

Suttering Butcher does stutter edits and glitch on an audio file.

**Note:** Only the first minute of `audio_file` will be processed.

<hr size="1" color="#666"/>

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisities.

import os
from google.colab import output
force_setup = False

pip_packages = 'pysoundfile'

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  %cd /content/
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if not os.path.isdir('/content/drive') and force_setup == False:
  from google.colab import drive
  drive.mount('/content/drive')

# Drive symlink
if not os.path.isdir('/content/mydrive') and force_setup == False:
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root_set = True
drive_root = '/content/mydrive/'

dir_tmp = '/content/tmp/'
create_dirs([dir_tmp])
last_data_file = ''

output.clear()
op(c.ok, 'Setup finished.')

##------------------------------------------------------------
##
## Imports & Defs
##
##------------------------------------------------------------

import soundfile
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
nper = np.seterr(divide = 'ignore') 




In [ ]:
#@markdown <small>Path to a WAV file located in your Google Drive.</small>
audio_file = "" #@param {type:"string"}
#@markdown <small>Minimum amplitude to stutter.</small>
y_threshold = 0.404 #@param {type:"slider", min:0.001, max:1, step:0.001}
#@markdown <small>Minimum stutter duration _(milliseconds)_. Reduces stutter by disregarding stutters shorter than this.</small>
x_threshold = 13 #@param {type:"slider", min:1, max:1000, step:1}
#@markdown <small>Maximum frequency to stutter _(hertz)_. Reduces stutter by tuning out frequencies above this.</small>
λ_threshold = 920 #@param {type:"slider", min:50, max:20000, step:10}
#@markdown <small>Size of single grain _(spatial periods)_.</small>
grain_size = 2 #@param {type:"slider", min:1, max:20, step:1}
#@markdown <small>Number of times a grain is repeated in a stutter.</small>
repeat_factor = 2 #@param {type:"slider", min:1, max:20, step:1}
#@markdown <small>Randomly vary `repeat_factor` (above) this much.</small>
repeat_variation = 1 #@param {type:"slider", min:0, max:10, step:1}


#@markdown <small>Reduce overall number of stutters and glitch, from whatever above settings otherwise actually produce.</small>
amount = -79 #@param {type:"slider", min:-100, max:0, step:1}
#@markdown <small>Increase crazy.</small>
glitch = 0 #@param {type:"slider", min:0, max:4, step:1}

# # #@markdown ##Advanced settings
# # #@markdown <small>Reduce clicks in stutters. Adds ~1 ms fade in/out **if** start/end sample is too far from 0.</small>
# # click_reduction = False #@param {type:"boolean"}
# # #@markdown <small>Grain fade in/out _(milliseconds)_.</small>
# # grain_fade = 0 #@param {type:"slider", min:0, max:3, step:1}
# # #@markdown <small>Output mono.</small>
# # mono = False #@param {type:"boolean"}
# # #@markdown <small>Tune all stutters to given note.</small>
# # autotune_to = "G3" #@param {type:"string"}
# # #@markdown <small>Stutter probably won't work well if audio has DC offset. Try to fix:</small>
# # dc_repair = False #@param {type:"boolean"}

#@markdown <hr color="#555" size="1">

#@markdown ##Saving
#@markdown <small>Save all stuttered audio to Drive.</small>
save_to_drive = False #@param {type:"boolean"}
#@markdown <small>Files will be saved in `audio_file`'s encolsing directory, if left empty.</small>
output_dir = "" #@param {type:"string"}

# Override advanced settings
yy = y_threshold
xx = x_threshold
y_threshold = xx
x_threshold = yy

click_reduction = True
autotune_to = 'None'
grain_fade = 44
dc_repair = False
randomness = repeat_variation

# Globals & adjustments
global_sr = 44100
lps_threshold = λ_threshold
yr_threshold = librosa.time_to_samples(y_threshold/1000, sr=global_sr)
global_fade = librosa.time_to_samples(grain_fade/1000, sr=global_sr)
y_threshold = 10
amount = (amount+100)/100
input_clip = 60
mono = False

# Random & Glitch
random_odds = randomness/100
autotune_odds = 0

if glitch > 0:
  yr_threshold = 1
  click_reduction = False
  autotune_odds = 0.4

audio_file = fix_path(drive_root+audio_file)
f_output_dir = fix_path(drive_root+output_dir)

butter = False
polarity = False
waves = grain_size
xtsca = [0.001, 1]
lpsca = [50, 20000]
gssca = [1, 20]
rfsca = [1, 50]
amsca = [0.01, 1]
notes = 'C2 D2 E2 F2 G2 A2 B2 C3 D3 E3 F3 G3 A3 B3 C4 D4 E4 F4 G4 A4 B4 C5 D5 E5 F5 G5 A5 B5 C6 D6 E6 F6 G6 A6 B6'
notes = notes.split()

waves = grain_size
repeat = repeat_factor

if dc_repair == True:
  butter = True

import librosa, scipy, random
import numpy as np
import matplotlib.pyplot as plt

lps = scipy.signal.butter(10, lps_threshold, 'lp', fs=global_sr, output='sos')
hps = scipy.signal.butter(10, 15, 'hp', fs=global_sr, output='sos')

mono_audio, sr = librosa.load(audio_file, sr=global_sr, duration=input_clip, mono=True)
stereo_audio, sr = librosa.load(audio_file, sr=global_sr, duration=input_clip, mono=False)
mono_lpa = librosa.util.normalize(scipy.signal.sosfilt(lps, mono_audio))
mono_audio = librosa.util.normalize(mono_audio)
plt.rcParams['figure.figsize'] = [25, 6]
plt.rcParams.update({"axes.facecolor": "black"})

def rnd(min, max, is_int=True):
  if is_int == True:
    return random.randint(round(min), round(max))
  else:
    return random.uniform(min, max)

def rnd_int(min, max):
  return random.randint(min, max)
  
def rndmz(val, scale, is_int=True, maximize=False):
  global randomness
  use_rand = randomness
  if maximize == True:
    use_rand = 100
  if use_rand > 0:
    randy = use_rand/100
    min = scale[0]
    max = scale[1]
    return rnd(min+(val-(val*randy)), max-(val+(val*randy)), is_int)
  else:
    return val

def merge_channels(left_data, right_data):
  return np.array([left_data, right_data])

def split_channels(audio_data):
  return audio_data[0], audio_data[1]

def swf(sig1, sig2='', peaks=[], rnd=False, sr=global_sr):
  #yellowgreen, salmon
  duration = len(sig1)/sr
  time = np.arange(0,duration,1/sr)
  plt.rcParams.update({"axes.facecolor": "black"})
  plt.ylim(-1, 1)
  if rnd==True:
    c = np.random.rand(3)
  else:
    c = '#00ffdd'
  plt.plot(time, sig1, color=c, linewidth=0.3, alpha=1)
  if len(peaks) > 0:
    for i, peak_set in enumerate(peaks):
      #print(i, peak_set)
      if i == len(peaks)-1:
        c = '#f3d'
        prio = .65
        lw = 0.7
      else:
        c = '#fff'
        prio = .2
        lw = 0.4
      for peak in peak_set:
        plt.axvline(x=peak/sr, color=c, linewidth=lw, alpha=prio)
  if sig2 != '':
    plt.plot(time, sig2, color=np.random.rand(3), linewidth=0.3, alpha=0.55)
  plt.show()

def detect_pitch(audio_data, t=0, sr=global_sr):
  pitches, magnitudes = librosa.core.piptrack(y=audio_data, sr=sr, fmin=50, fmax=900)
  index = magnitudes[:, t].argmax()
  pitch = pitches[index, t]
  return pitch

def autotune_audio(audio_data, to_note='C3', sr=global_sr, t=0):
  pitch = detect_pitch(audio_data, t, sr=sr)
  source_note = round(librosa.hz_to_midi(pitch))
  target_note = librosa.note_to_midi(to_note)
  if source_note > 0:
    diff = round(target_note-source_note)
    oct = 12 if diff > 0 else -12
    octs = math.floor(diff/oct)
    if octs > 0:
      diff = diff-octs*oct
    elif octs < 0:
      diff = diff+octs*oct
    if diff < -6:
      if octs < 0:
        diff = octs*oct-diff
      else:
        diff = oct-diff
    elif diff > 6:
      if octs > 0:
        diff = octs*oct+diff
      else:
        diff = oct-diff 
    tuned = librosa.effects.pitch_shift(audio_data, sr=sr, n_steps=diff, bins_per_octave=12)
  else:
    tuned = audio_data
  return tuned

def fade_audio(audio_data, fade_in=global_fade, fade_out=global_fade, sr=global_sr):
  if fade_in > 0 or fade_out > 0:
    a_duration = librosa.get_duration(audio_data, sr=sr)
    if fade_in > 0:
      in_y = audio_data[0:fade_in]
      fade = [ i/len(in_y)*smp for i, smp in enumerate(in_y) ]
      tail_start = fade_in
      tail = audio_data[tail_start:]
      audio_data = np.concatenate([fade, tail])
    if fade_out > 0:
      out_y = audio_data[fade_out:]
      fade = [ smp-(i/len(out_y)*smp) for i, smp in enumerate(out_y) ]
      head_start = fade_out
      head = audio_data[:head_start]
      audio_data = np.concatenate([head, fade])
  return audio_data

def declick(data, samples=50, to=None):
  head = data[:len(data)-samples]
  tail = data[len(data)-samples:]
  if to == None:
    to = head[0]
  linear = np.linspace(tail[0], to, samples)
  new_tail = []
  for i, smp in enumerate(tail):
    new_point = smp + (i/samples*linear[i]) - (smp * (i/samples))
    new_tail.append(new_point)
  return np.concatenate([head, new_tail]).ravel().tolist()

def grain(audio, x, y):
  global amount, xtsca, glitch, repeat, rfsca
  zero_points = []
  affected_points = []
  rndx = []
  rndx2 = []
  note = []
  last_point = 0
  las_val = 0
  drop_limit = 0.005
  repeats = []
  if glitch > 0:
    drop_limit = 0
  for i, mg in enumerate(audio):
    if i > y and i < len(audio)-y:
      if (round(mg, 2) == 0 and i > last_point and audio[i-1] < drop_limit and audio[i+1] > drop_limit and glitch <= 2) or (glitch == 3 and mg > x and i > last_point+y and odds(0.05) == True) or ((glitch == 4 and mg > x and odds(0.05) == True)):
        zero_points.append(i)
        rept = rndmz(repeat, rfsca) if randomness > 0 or glitch >= 3 else repeat
        if glitch > 1 and odds(0.5):
          rept = 1+math.ceil(rept/4)
        repeats.append(rept)
        rndx.append(odds(0.2))
        rndx2.append(odds(0.2))
        note.append(random.choice(notes))
        last_point = i
        las_val = mg
  durations = np.append(np.diff(zero_points), len(audio)-zero_points[-1]) if len(zero_points) else 0
  durations = durations.tolist()
  for i, start_smp in enumerate(zero_points):
    start = start_smp
    end = start+durations[i]
    audio_grain = audio[start:end]
    if max(audio_grain) > x and i < len(zero_points)-1 and odds(amount) == True:
      affected_points.append(start)
  durations = np.array(durations).ravel().tolist()
  return zero_points, durations, affected_points, repeats, note, rndx, rndx2

def get_stutter_pitches(audio, inv_points, inv_durations):
  pitches = []
  for i, inv in enumerate(inv_points):
    end = inv+inv_durations[i]
    pitches.append(detect_pitch(audio[inv:end]))
  return pitches

def fade_pro_re_nata(audio_data, samples=44):
    fade_in = samples if round(audio_data[0], 2) != 0 else 0
    fade_out = samples if round(audio_data[-1], 2) != 0 else 0
    return fade_audio(audio_data, fade_in, fade_out)

def stutter(audio, starts, durations, affected_starts, repeats, note, rndx, rndx2):
  global waves, repeat, amount
  global glitch, yr_threshold
  global new_starts
  autotune = False
  reps = []
  #reps.append(audio[0:starts[0]])
  for i, start_smp in enumerate(starts[:len(starts)-1]):
    start = starts[i]
    end = start+durations[i]
    audio_grain = audio[start:end]
    if start_smp in affected_starts and len(audio_grain)*repeats[i] >= yr_threshold:
      if glitch >= 2:
        if rndx[i] == True:
          audio_grain = (audio_grain, -audio_grain) if i % 2 else (-audio_grain, audio_grain)
          audio_grain = np.concatenate(audio_grain, axis=0)
        rep = 4 if durations[i] > 1000 else round(repeats[i]/2)
        audio_grain = np.tile(audio_grain, rep)
      if click_reduction == True and glitch == 0 and start != end:
        audio_grain = declick(audio_grain)
      rep = repeats[i]
      if click_reduction == True and glitch == 0 and start != end:
        declicked_audio_grain = declick(audio_grain)
        last_audio_grain = declick(audio_grain, to=audio[end+1])
        looped_audio_grain = np.tile(declicked_audio_grain, rep-1)
        audio_grain = np.concatenate([looped_audio_grain, last_audio_grain])
      if glitch == 1:
        audio_grain = np.tile(audio_grain, rep)
      if glitch >= 2 and rndx2[i] == True and len(audio_grain) > 660:
        audio_grain = autotune_audio(audio_grain, random.choice(notes))
    reps.append(audio_grain)
  return np.concatenate(reps)

starts, durations, affected_starts, repeats, note, rndx, rndx2 = grain(mono_lpa, x_threshold, y_threshold)
swf(mono_lpa, peaks=[affected_starts])

if waves > 1:
  starts = starts[0::waves]
  durations_sum = []
  for i in range(waves):
    durations_sum.append( durations[i::waves] )
  durations = [sum(x) for x in zip(*durations_sum)]

if mono == True:
  stereo_audio = merge_channels(mono_audio, mono_audio)

stuttered = np.array([stutter(chan, starts, durations, affected_starts, repeats, note, rndx, rndx2) for chan in split_channels(stereo_audio)], dtype=np.float64)

swf(stuttered[0], stuttered[1])
audio_player(stuttered)

if save_to_drive == True:
  print('')
  if save_to_drive == True and output_dir == '':
    f_output_dir = path_dir(audio_file)
    op(c.fail, 'Output directory was not set.')
  if save_to_drive == True and not os.path.isdir(f_output_dir):
    f_output_dir = path_dir(audio_file)
    op(c.fail, 'Output directory was not found.')
  save_as = f_output_dir+'stuttered_'+basename(audio_file)+'__'+rnd_str(6)+'.wav'
  soundfile.write(save_as, stuttered.T, global_sr)
  op(c.ok, 'File saved as', save_as.replace(drive_root, ''))

